In [1]:
from collections import namedtuple
And = namedtuple('And', 'a b')
Or = namedtuple('Or', 'a b')
Not = namedtuple('Not', 'a')
Var = namedtuple('Var', 'name')
Const = namedtuple('Const', 'val')

In [2]:
type(And(True, False)) == And

True

In [3]:
# expects env to be a dictionary and expr to be a chain of named tuples
def logeval(env, expr): 
    if type(expr) == And:
        return logeval(env, expr.a) and logeval(env, expr.b)
    elif type(expr) == Or:
        return logeval(env, expr.a) or logeval(env, expr.b)
    elif type(expr) == Not:
        return not logeval(env, expr.a)
    elif type(expr) == Var:
        return env[expr.name]
    else:
        print("ill formed query")

        
        

In [4]:
env = {'a': True, 'b': False}
expr1 = Not(Var('a'))
logeval(env, expr1)

False

In [5]:
env = {'a': True, 'b': True}
expr1 = And(Var('a'), Var('b'))
logeval(env, expr1)

True

In [6]:
bool((1 << 1) & 7)

True

In [16]:
expr2 = And(Var('a'), Not(Var('a')))

In [7]:
# an all combo generator
def allcombos(varlist):
    env = {}
    for j in range(2**len(varlist)):
        for varnum, var in enumerate(varlist):
            env[var] = bool((1 << varnum) & j)
        yield env
    

In [8]:
list(map(print, allcombos(['a','b'])))

{'b': False, 'a': False}
{'b': False, 'a': True}
{'b': True, 'a': False}
{'b': True, 'a': True}


[None, None, None, None]

In [9]:
def simpleSAT(varlist, expr):
    for env in allcombos(varlist):
        if logeval(env, expr):
            return env
    return None


In [10]:
simpleSAT(['a','b'], expr1)

{'a': True, 'b': True}

In [20]:
def findunbound(varlist, env):
    for var in varlist:
        if var not in env:
            return var
    return None

def recurseSAT0(env, varlist, expr):
    unboundvar = findunbound(varlist, env)
    if unboundvar:
        env[unboundvar] = True #Try true assignment
        if recurseSAT0(env, varlist, expr):
            return True
        else:
            env[unboundvar] = False 
            if recurseSAT0(env, varlist, expr):
                return True
            else:
                env.pop(unboundvar, None) #return env to previous form on backtracking
                return False
    else: #All variables assigned
        return logeval(env, expr)

def recurseSAT(varlist, expr):
    env = {}
    if recurseSAT0(env, varlist, expr):
        return env
    else:
        return "UnSAT"
    


In [14]:
recurseSAT(['a','b'], expr1)

{'a': True, 'b': True}

In [21]:
recurseSAT(['a','b'], expr2)

'UnSAT'

Check out walkSat

In [ ]:
# expects env to be a dictionary and expr to be a chain of named tuples
def logreplace(env, expr): 
    if type(expr) == And:
        return And(logreplace(env, expr.a), logreplace(env, expr.b))
    elif type(expr) == Or:
        return Or(logreplace(env, expr.a), logreplace(env, expr.b))
    elif type(expr) == Not:
        return not Not(logreplace(env, expr.a))
    elif type(expr) == Var:
        if expr.name in env:
            return Const(env[expr.name])
        else:
            return expr
    elif type(expr) == Const:
        return expr
    else:
        print("ill formed query")

God. These are SUCH folds.

In [ ]:
def logsimp(env, expr):
    if type(expr) == And:
        if type(expr.a) == Const:
            
        elif type(expr.b) == Const:
            
        return And(logreplace(env, expr.a), logreplace(env, expr.b))
    elif type(expr) == Or:
        return Or(logreplace(env, expr.a), logreplace(env, expr.b))
    elif type(expr) == Not:
        return not Not(logreplace(env, expr.a))
    elif type(expr) == Var:
        if expr.name in env:
            return Const(env[expr.name])
        else:
            return expr
    elif type(expr) == Const:
        return expr
    else:
        print("ill formed query")